In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
# from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re
from nltk.stem import WordNetLemmatizer


In [2]:
df_hash = {
#     "cooking": pd.read_csv("cooking.csv"),
#     "crypto": pd.read_csv("crypto.csv"),
    "robotics": pd.read_csv("robotics.csv"),
#     "biology": pd.read_csv("biology.csv"),
#     "travel": pd.read_csv("travel.csv"),
#     "diy": pd.read_csv("diy.csv"),
}

print(df_hash["robotics"].iloc[1])

id                                                         2
title      How can I modify a low cost hobby servo to run...
content    <p>I've got some hobby servos (<a href="http:/...
tags                                         control rcservo
Name: 1, dtype: object


In [3]:
df_hash['robotics'].columns.values

array(['id', 'title', 'content', 'tags'], dtype=object)

In [4]:
# biologyTable.shape
print(df_hash["robotics"].shape)
print df_hash["robotics"].tags.head(5)

(2771, 4)
0                          soccer control
1                         control rcservo
2                               gait walk
3    microcontroller arduino raspberry-pi
4                     motion-planning rrt
Name: tags, dtype: object


In [5]:
import nltk
# nltk.download()
from nltk.corpus import stopwords
# In Python, searching a set is much faster than searching a list, so convert the stop words to a set
stops = set(stopwords.words("english"))  

https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words

In [6]:
def separate_tags(tags):
#     print tags
    letters_only = re.sub("[^a-zA-Z]", " ", tags) 
#     return map( lambda x: [x], letters_only.lower().split() )
    return letters_only.lower().split()

# print separate_tags("hello googbye13 42 HI")

In [7]:
def strip_html(raw_html):
    if raw_html:
        # remove html tags
        soup = BeautifulSoup(raw_html, "html.parser")

        [s.extract() for s in soup(['pre', 'code'])]
            
        question_text = soup.get_text()
        
        # remove everything but letters
        letters_only = re.sub("[^a-zA-Z]", " ", question_text) 
        
            
        # normalize case
        words = letters_only.lower().split()   

        # remove stopwords         
        meaningful_words = [w for w in words if not w in stops] 
        
        wordnet_lemmatizer = WordNetLemmatizer()
        meaningful_word_stems = map(lambda x: wordnet_lemmatizer.lemmatize(x) , meaningful_words)
        return( " ".join( meaningful_word_stems )) 
    else:
        return ""
    
# strip_html(df_hash['robotics'].content[0])

In [8]:
print "Cleaning and parsing the training set of tags for each topic...\n"

for topic, df in df_hash.iteritems():
    print topic
    print "'s size is "
    
    df.tags = df.tags.apply(lambda x: separate_tags(x) )
    df.tags.replace(['', ' '], np.nan, inplace=True)
    df.dropna(subset=['tags'], inplace=True)
    print df.tags

Cleaning and parsing the training set of tags for each topic...

robotics
's size is 
0                                       [soccer, control]
1                                      [control, rcservo]
2                                            [gait, walk]
3               [microcontroller, arduino, raspberry, pi]
4                                 [motion, planning, rrt]
5                                    [software, platform]
6                                     [software, circuit]
7                [odometry, localization, kalman, filter]
8                                              [untagged]
9                                     [soccer, mechanism]
10                     [computer, vision, wheeled, robot]
11                                           [quadcopter]
12                                               [servos]
13                          [localization, mobile, robot]
14                                        [kinect, input]
15                                          

In [9]:
print df_hash["robotics"].shape

(2771, 4)


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class QuestionStripper(BaseEstimator, TransformerMixin):

    def __init__(self):
        self = self
#         self.df = df  # e.g. pass in a column name to extract
#         self.topic_size = df.size
        
    def transform(self, X, y=None):
#         return do_something_to(X, self.vars)  # where the actual feature extraction happens
#         print "'s size is "
#         print self.topic_size
        return X.apply(lambda x: strip_html(x) )
        
    
    def fit(self, X, y=None):
        return self  # generally does nothing

In [11]:
print df_hash["robotics"].tags.head(5)
print df_hash["robotics"].shape


0                            [soccer, control]
1                           [control, rcservo]
2                                 [gait, walk]
3    [microcontroller, arduino, raspberry, pi]
4                      [motion, planning, rrt]
Name: tags, dtype: object
(2771, 4)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier, OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import MultiLabelBinarizer
# import sklearn.grid_search
from sklearn.model_selection import GridSearchCV


TfidfTransformer = TfidfVectorizer(analyzer = "word",   \
                         tokenizer = None,    \
                         preprocessor = None, \
                         stop_words = None,   \
                         max_features = 5000) 
        




        
# ('param_searcm', gridsearchCV)] )

steps = [('strip_questions', QuestionStripper()),
         ('tfidf', TfidfTransformer ),
         ('svc', OneVsRestClassifier(LinearSVC()))]
#         ('output_classifier' , MultiOutputClassifier)]



pipeline = Pipeline(steps)


# Set the parameters by cross-validation
# parameters = {'svc__estimator__kernel': ['rbf', 'poly']}
#               'svc__estimator__degree':[1, 2, 3] }

# parameters = dict(svc__estimator__C=np.logspace(-5,0,5), svc__estimator__gamma=np.logspace(-2, 2, 10))

#     print("Best parameters set found on development set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on development set:")
#     print()
#     means = clf.cv_results_['mean_test_score']
#     stds = clf.cv_results_['std_test_score']
#     for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#         print("%0.3f (+/-%0.03f) for %r"
#               % (mean, std * 2, params))
        
        
  
for topic, df in df_hash.iteritems():
    print "splitting"
    mlb = MultiLabelBinarizer()
    Y = mlb.fit_transform(df.tags)
#     print Y.shape

    print df.tags.head(5)
    X_train, X_test, y_train, y_test = train_test_split(df.content,\
                                                        Y, \
                                                        test_size=0.33,\
                                                        random_state=42)
     
    scorer = make_scorer(f1_score, average="weighted")
#     Y_test = mlb.fit_transform(y_test)
#     scores = ['precision', 'recall']

#     for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     cv = GridSearchCV( pipeline, scoring='f1')
#     cv = GridSearchCV( pipeline, param_grid = parameters, scoring='f1')
#                            scoring='%s_macro' % score)

    pipeline.fit( X_train, y_train )
    y_prediction = pipeline.predict( X_test )
    y_test_inv = mlb.inverse_transform(y_test)
    y_pred_inv = mlb.inverse_transform(y_prediction)

#     print y_test_inv[0:4]
#     print y_pred_inv[0:4]

#     for item, labels in zip(X_test, y_pred_inv):
#         print('{0} => {1}'.format(item, ', '.join(labels)))

    
#     report = classification_report( y_test, y_prediction )
#     print(report)
    
#     print cv.best_score_
#     print cv.best_params_
    


splitting
0                            [soccer, control]
1                           [control, rcservo]
2                                 [gait, walk]
3    [microcontroller, arduino, raspberry, pi]
4                      [motion, planning, rrt]
Name: tags, dtype: object


/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 43 is present in all training examples.
  str(classes[c]))
/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 51 is present in all training examples.
  str(classes[c]))
/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 91 is present in all training examples.
  str(classes[c]))
/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 107 is present in all training examples.
  str(classes[c]))
/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 204 is present in all training examples.
  str(cla

In [13]:
index = 0
for actual, pred in zip(y_test_inv, y_pred_inv):
        print('index: {0} :: {1} => {2}'.format(index, actual, ', '.join(pred)))
        index = index +1

        

index: 0 :: ('laser', 'mobile', 'probability', 'robot', 'slam') => 
index: 1 :: ('arm', 'gearing', 'motor', 'robotic') => motor, torque
index: 2 :: ('mobile', 'power', 'quadcopter', 'robot') => quadcopter
index: 3 :: ('balance', 'dynamics') => 
index: 4 :: ('arm', 'kinect', 'robotic', 'sensors', 'stereo', 'vision') => kinect, vision
index: 5 :: ('create', 'irobot') => create, irobot
index: 6 :: ('inverse', 'kinematics') => inverse, kinematics
index: 7 :: ('control', 'create', 'irobot', 'motor') => create, irobot
index: 8 :: ('arduino', 'arm', 'inverse', 'kinematics', 'learning', 'machine', 'robotic') => arm, robotic
index: 9 :: ('calibration', 'inverse', 'kinematics') => arm, inverse, kinematics
index: 10 :: ('magnetometer', 'orientation') => 
index: 11 :: ('current',) => 
index: 12 :: ('cameras', 'control', 'localization', 'ros', 'sensors') => 
index: 13 :: ('filter', 'kalman') => filter, kalman
index: 14 :: ('mobile', 'robot') => 
index: 15 :: ('embedded', 'microcontroller', 'real', 

In [14]:
print y_pred_inv

[(), ('motor', 'torque'), ('quadcopter',), (), ('kinect', 'vision'), ('create', 'irobot'), ('inverse', 'kinematics'), ('create', 'irobot'), ('arm', 'robotic'), ('arm', 'inverse', 'kinematics'), (), (), (), ('filter', 'kalman'), (), (), (), (), (), (), ('vision',), ('robot',), ('arduino', 'pi', 'raspberry'), ('create', 'irobot'), ('esc', 'quadcopter'), ('filter', 'particle'), ('gyroscope', 'sensors'), (), ('filter', 'kalman'), (), (), ('battery', 'motor', 'power'), ('ros',), ('arm', 'mobile', 'motion', 'planning', 'robot', 'robotic'), (), (), ('motor',), ('filter', 'localization', 'particle'), ('kinect', 'vision'), (), ('kinematics',), (), ('motor', 'pid'), ('pi', 'raspberry'), (), ('arduino',), (), (), (), ('cameras', 'computer', 'stereo', 'vision'), ('sensors',), ('pid',), (), ('motor', 'pid'), ('robot',), (), ('jacobian', 'kinematics'), ('arduino', 'pi', 'raspberry'), ('battery',), ('control', 'dynamics'), (), ('sensors',), ('quadcopter',), ('mobile', 'robot', 'wheeled'), (), ('motor

https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/

In [15]:
# Numpy arrays are easy to work with, so convert the result to an 
# array
# train_data_features = train_data_features.toarray()

In [16]:
# print train_data_features.shape
# print "4"